# Sinabs

Sinabs is a deep learning library for spiking neural networks which is based on PyTorch and focuses on fast training as well as inference on neuromorphic hardware.

`sinabs.to_nir` and `sinabs.from_nir` methods allow you to seemlessly navigate between `nir` and `sinabs`. Once your model is in sinabs, you can use this model to train or directly deploy your models to Speck/DynapCNN. 


## Import a NIR graph to Sinabs

In [ ]:
import torch
from sinabs import from_nir

import nir

# Create a NIR graph
affine_weights = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
affine_bias = torch.tensor([1.0, 2.0])
li_tau = torch.tensor([0.9, 0.8])
li_r = torch.tensor([1.0, 1.0])
li_v_leak = torch.tensor([0.0, 0.0])
nir_network = nir.NIRGraph.from_list(
    nir.Affine(affine_weights, affine_bias), nir.LI(li_tau, li_r, li_v_leak)
)

# Create Sinabs model from NIR graph.
# You need to define the batch size because Sinabs will use Squeeze
# versions of layers by default.
sinabs_model = from_nir(nir_network, batch_size=10)
print(sinabs_model)

## Export a NIR graph from Sinabs

In [ ]:
import sinabs.layers as sl
import torch
import torch.nn as nn
from sinabs import from_nir, to_nir

batch_size = 4

# Create Sinabs model
orig_model = nn.Sequential(
    torch.nn.Linear(10, 2),
    sl.ExpLeakSqueeze(tau_mem=10.0, batch_size=batch_size),
    sl.LIFSqueeze(tau_mem=10.0, batch_size=batch_size),
    torch.nn.Linear(2, 1),
)

# Convert model to NIR graph with a random input of representative shape
nir_graph = to_nir(orig_model, torch.randn(batch_size, 10))
print(nir_graph)


# Reload sinabs model from NIR
sinabs_model = from_nir(nir_graph, batch_size)
